In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, Event
import pandas
import plotly.graph_objs as go
import time
import numpy as np
from scipy.sparse.linalg import svds
from surprise import Reader, Dataset, SVD, evaluate
import pickle
import time
import heapq

In [2]:


def mov_list_for_all_users(dr,dm):



#     dr = pandas.read_csv('ratings.csv')
#     dm = pandas.read_csv('movies.csv')


    dr.head(10)

    n_users = dr.userId.unique().shape[0]
    n_movies = dr.movieId.unique().shape[0]
    print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

    mov_ids = dm['movieId']
    mov_dict = {}
    i=1
    for id1 in mov_ids:
        mov_dict[id1]=i
        i += 1
    len( mov_dict)

    # dr.replace({"movieId": mov_dict})

    rats = dr.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
    rats.head()


    A = rats.as_matrix()
    A


    users_average_rating = np.mean(A, axis = 1)
    # users_average_rating
    # len(users_average_rating)

    rat_diff = A - users_average_rating.reshape(-1,1)
    # rat_diff

    # sparsity = round(1.0 - len(dr) / float(n_users * n_movies), 3)
    # print ('The sparsity level of MovieLens1M dataset is ' +  str(sparsity * 100) + '%')

    U, sigma, Vt = svds(rat_diff, k = 20)



    sigma_mat = np.diag(sigma)

    # sigma_mat

    rat_predicted = np.dot(np.dot(U, sigma_mat), Vt) + users_average_rating.reshape(-1, 1)
    
    rec_mov_all_user = {}
    
    
    user_id_list = dr.userId.unique().tolist()
    
    for user_id1 in user_id_list:
        
#         print(user_id1)

    
    

        user_id = int(user_id1)-1
        rec_mov = []


        rat_predicted_user = rat_predicted[user_id,:]

        # rat_predicted_user_list = rat_predicted_user.tolist()

        # rat_predicted_user_list = rat_predicted_user


        dp = dr.loc[dr['userId'] == int(user_id1)]

        user_mvs_ids = dp.iloc[:,1].tolist()


        for i in range(0,len(rat_predicted_user)):
            if i in user_mvs_ids:
                rat_predicted_user[i] =  -1000

        rat_predicted_user



#         top_5_idx = np.argsort(-rat_predicted_user)[-10:]
        
        top_5_idx = rat_predicted_user.argsort()[-20:][::-1]

        top_5_idx
        top_5_id_list = top_5_idx.tolist()

        rec_mov_name = []
        
        itt = 0


        for mov_id in top_5_id_list:
            
            itt += 1

            dummy = dm.loc[dm['movieId'] == mov_id]
            
            if(len(dummy.index)>0):

                rec_mov_name.append(dummy.iloc[0,1])
#             print(dm.loc[dm['movieId'] == mov_id])
            if itt > 10:
                break;
            
        rec_mov_all_user[user_id1] = rec_mov_name

        # rec_mov_df = dm.loc[dm['movieId'] in top_5_id_list]

    #     print(rec_mov_name)
    return rec_mov_all_user

In [3]:
# dr = pandas.read_csv('ratings.csv')
# dm = pandas.read_csv('movies.csv')


# user_id_list = dr.userId.unique().tolist()
# user_id_list

In [4]:
dr = pandas.read_csv('ratings.csv')
dm = pandas.read_csv('movies.csv')
mov_list_all = mov_list_for_all_users(dr,dm)

# print (mov_list_all)

Number of users = 610 | Number of movies = 9724


C:\Users\anik\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [5]:
print (mov_list_all[240])

["What's Eating Gilbert Grape (1993)", 'Chungking Express (Chung Hing sam lam) (1994)', 'Hate (Haine, La) (1995)', 'Orlando (1992)', 'Swimming with Sharks (1995)', 'Being Human (1993)', 'Philadelphia (1993)', 'Piano, The (1993)', 'Secret of Roan Inish, The (1994)']


In [6]:
with open('mov_list_svd.pkl', 'wb') as handle:
    pickle.dump(mov_list_all, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [7]:
def get_movie_ids_from_names(dm,mov_list):
    ids = []
    for mv in mov_list:
        dp = dm.loc[dm['title']==mv]
        iden = dp.iloc[0,0]
        ids.append(iden)
    return ids

In [11]:
# for evaluation 

def get_user_movie_types(dr,dm,user_id):
    movie_type = {}
    types = []
    dp = dr.loc[dr['userId'] == int(user_id)]
    dpp = dp.loc[dp['rating'] == 5]
    
    mov_list = dpp.iloc[:,1].tolist()
    
    for mv in mov_list:
        dummy = dm.loc[dm['movieId'] == int(mv)]
        movie_type[dummy.iloc[0,1]] = dummy.iloc[0,2]
        arrays = dummy.iloc[0,2].split("|")
        for item in arrays:
            types.append(item)
#         types.append()
        
    return movie_type,types
    
def movie_types(dm,mov_list):
    types = []
    for mv in mov_list:
        dp = dm.loc[dm['title']==mv]
        arrays = dp.iloc[0,2].split("|")
        for item in arrays:
            types.append(item)
        
    return types
   
def movie_types_by_id(dm,mov_list):
    types = []
    for mv in mov_list:
        dp = dm.loc[dm['movieId']==mv]
        arrays = dp.iloc[0,2].split("|")
        for item in arrays:
            types.append(item)
        
    return types    


            
    


In [12]:
# for old user 
user_type = get_user_movie_types(dr,dm,240)
# print(user_type[0])
user_movie_types = set(user_type[1])
print(set(user_type[1]))

mov_list = ["What's Eating Gilbert Grape (1993)", 'Chungking Express (Chung Hing sam lam) (1994)', 'Hate (Haine, La) (1995)', 'Orlando (1992)', 'Swimming with Sharks (1995)', 'Being Human (1993)', 'Philadelphia (1993)', 'Piano, The (1993)', 'Secret of Roan Inish, The (1994)']
mov_types = set(movie_types(dm,mov_list))
print(set(mov_types))
print("......................................")
print ("Common types of watched and recommended movies.")
print(user_movie_types.intersection(mov_types))

{'Musical', 'Action', 'War', 'Adventure', 'Fantasy', 'Romance', 'Animation', 'Thriller', 'Crime', 'Sci-Fi', 'Horror', 'Comedy', 'Children', 'Drama', 'Western'}
{'Crime', 'Drama', 'Mystery', 'Comedy', 'Children', 'Romance', 'Fantasy'}
......................................
Common types of watched and recommended movies.
{'Romance', 'Fantasy', 'Crime', 'Comedy', 'Children', 'Drama'}
